In [ ]:
from pathlib import Path

from pcd_dataset.kitti_dataset import KittiDataset

dataset_path = 'dataset/'
sequence = '00'
sam_labels_path = Path.cwd().joinpath('pipeline', 'vfm-labels', 'sam', sequence)

kitti = KittiDataset(dataset_path, sequence, sam_labels_path)

In [ ]:
from mapping.simple_mapping import SimpleMapping

sm = SimpleMapping(kitti)
labeled_pcds = sm.get_combined_labeled_point_clouds('cam2', 20, 25)